In [ ]:
from models import *
import numpy as np
import torch
import torch.nn as nn
from torchvision.transforms import Compose, ToTensor, Resize, CenterCrop
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from eval_utils import evaluate_OOD_detection, evaluate_ID_detection
import json
import pickle

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [2]:
###Modify Me according to your paths###
# ckpt_dir = "checkpoints/"
ood_data_dir = "../data/OOD/"
id_data_dir = "../data/ID/"

# prepare data

500 ID images and 500 OOD images
target for ID images is 1 and target for OOD images is 0. (binary OOD detection)

In [3]:
ori_preprocess = Compose([
        Resize((224), interpolation=Image.BICUBIC),
        CenterCrop(size=(224, 224)),
        ToTensor()])

/home/mamooler/anaconda3/envs/sem_proj/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [4]:


class Image_Dataset(Dataset):
    def __init__(self, id_data_dir, ood_data_dir):
        self.id_data_dir = id_data_dir+"acorn squash"
        self.ood_data_dir = ood_data_dir
        self.data = []
        for file in os.listdir(self.id_data_dir)[:10]:
            self.data.append((self.id_data_dir+"/"+file, 0))
        # for file in os.listdir(self.ood_data_dir)[:1]:
        #     self.data.append((self.ood_data_dir+file, 1))
        print(self.data)

    def __getitem__(self, idx):
        
        img_path, target = self.data[idx]
        img = ori_preprocess(Image.open(img_path))


        return img, target

    def __len__(self):
        return len(self.data)



In [5]:
dataset = Image_Dataset(id_data_dir, ood_data_dir)
data_loader = DataLoader(dataset, batch_size = 1, shuffle= True)

thresholds = torch.range(0.5, 0.9, step=0.1)

[('../data/ID/acorn squash/295894360_57301e7f52.jpg', 0), ('../data/ID/acorn squash/3131799338_30cc952ab4.jpg', 0), ('../data/ID/acorn squash/2728402305_0423769588.jpg', 0), ('../data/ID/acorn squash/1481650104_cbe7276bb6.jpg', 0), ('../data/ID/acorn squash/3061250714_a132b0fc7e.jpg', 0), ('../data/ID/acorn squash/3219153054_5002a1d7a5.jpg', 0), ('../data/ID/acorn squash/2478437330_2a7d5626ae.jpg', 0), ('../data/ID/acorn squash/2982117320_133c332c66.jpg', 0), ('../data/ID/acorn squash/2758144776_ff013800b6.jpg', 0), ('../data/ID/acorn squash/2981258993_0542f8c00f.jpg', 0)]


/tmp/ipykernel_22323/1745201148.py:4: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  thresholds = torch.range(0.5, 0.9, step=0.1)


# ResNet

In [6]:
ood_threshold = 0.5
resnet_model = resnet.ResNet().to(device)
# the goal is to identify OOD samples

resnet_prc, resnet_rec, resnet_f1 = evaluate_OOD_detection(resnet_model, data_loader, thresholds, device)
print(resnet_prc, resnet_rec, resnet_f1)

/home/mamooler/anaconda3/envs/sem_proj/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /tmp/pip-req-build-1wfnpb1e/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/mamooler/MA3/VisInt/vit-vs-cnn/notebooks/models/resnet.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = self.softmax(logits)


tensor([[0.6000],
        [0.4000],
        [0.4000],
        [0.4000],
        [0.4000]]) tensor([[0.6000],
        [0.4000],
        [0.4000],
        [0.4000],
        [0.4000]]) tensor([[0.6000],
        [0.4000],
        [0.4000],
        [0.4000],
        [0.4000]])


# DeiT

In [7]:
deit_model = deit.DeiT().to(device)

deit_prc, deit_rec, deit_f1 = evaluate_OOD_detection(deit_model, data_loader, thresholds, device)
print(deit_prc, deit_rec, deit_f1)

/home/mamooler/MA3/VisInt/vit-vs-cnn/notebooks/models/deit.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = self.softmax(logits)


tensor([[0.6000],
        [0.3000],
        [0.2000],
        [0.0000],
        [0.0000]]) tensor([[0.6000],
        [0.3000],
        [0.2000],
        [0.0000],
        [0.0000]]) tensor([[0.6000],
        [0.3000],
        [0.2000],
        [0.0000],
        [0.0000]])


# ConvMixer

In [ ]:
convmixer_model = convmixer.ConvMixer().to(device)

convmixer_prc, convmixer_rec, convmixer_f1 = evaluate_OOD_detection(convmixer_model, data_loader, thresholds, device)
print(convmixer_prc, convmixer_rec, convmixer_f1)

# MLPMixer

In [ ]:
mlpmixer_model = mlpmixer.MLPMixer().to(device)

mlpmixer_prc, mlpmixer_rec, mlpmixer_f1 = evaluate_OOD_detection(mlpmixer_model, data_loader, thresholds, device)
print(mlpmixer_prc, mlpmixer_rec, mlpmixer_f1)

# ECA-ResNet

In [10]:
ecaresnet_model = ecaresnet.ECAResNet().to(device)
ecaresnet_prc, ecaresnet_rec, ecaresnet_f1 = evaluate_OOD_detection(ecaresnet_model, data_loader, thresholds, device)
print(ecaresnet_prc, ecaresnet_rec, ecaresnet_f1)

/home/mamooler/MA3/VisInt/vit-vs-cnn/notebooks/models/ecaresnet.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = self.softmax(logits)


tensor([[0.7000],
        [0.6000],
        [0.4000],
        [0.3000],
        [0.0000]]) tensor([[0.7000],
        [0.6000],
        [0.4000],
        [0.3000],
        [0.0000]]) tensor([[0.7000],
        [0.6000],
        [0.4000],
        [0.3000],
        [0.0000]])
